In [2]:
import numpy as np 
import h5py as h5 
import os
from BetaPose import data_io

matrix = np.arange(100*32*32*32).reshape((100,32,32,32))

targetval = np.arange(100)

# # print(matrix)
with h5.File("/tmp/mytestfile.h5", "w") as file1:
  print(matrix.shape)
  dset = file1.create_dataset("train_x", matrix.shape, dtype='i4', data=matrix, compression="gzip", maxshape=(None, 32,32,32))
  targetvals = file1.create_dataset("train_y", targetval.shape, dtype='i4', data=targetval, compression="gzip", maxshape=(None))

print("################")
hdf = data_io.hdf_operator("/tmp/mytestfile.h5")

# Entry Addition
hdf.append_entry("train_x", matrix)

# Entry Deletion
hdf.mask_entries("train_x", np.random.random(size=200)>0.1)
hdf.remove_entry("train_x", 10)

# Entry Alternation
hdf.alter_entry("train_x",1, (np.zeros((32,32,32))+1.5).astype(float))

hdf.draw_structure()


hdf.draw_structure()

datasets = hdf.list_datasets()
names = hdf.column_names("train_x")

hdf.close()


(100, 32, 32, 32)
################
Appended 100 entries to train_x
Created Dataset: train_x_temp
Successfully masked 173 entries; Shape: (200, 32, 32, 32) -> (173, 32, 32, 32)
Successfully Delete the entry 10; Shape: (173, 32, 32, 32) -> (172, 32, 32, 32)
####### HDF File Structure #######
$ /train_x             : Shape-(172, 32, 32, 32)
$ /train_y             : Shape-(100,)
##### END HDF File Structure #####
####### HDF File Structure #######
$ /train_x             : Shape-(172, 32, 32, 32)
$ /train_y             : Shape-(100,)
##### END HDF File Structure #####
None


In [3]:
from BetaPose import data_io
import h5py as h5
import pandas as pd
import numpy as np 
datatypes = [("atom_number", int), ("carbon_number", int), ("donor_number", int), ("acceptor_number", int),
             ("positive_charge", float), ("negative_charge", float), ("pseudo_lj", float), ("pseudo_elec", float),
             ("surface_area", float), ("volume", float), ("mean_radius", float), ("convex_ratio", float)]

hdf = data_io.hdf_operator("/tmp/test.h5", new=True)

print("Initialize datasets")
data1 = np.random.normal(size=(100, 12))
# data1 = np.asarray(data1, dtype=datatypes)
data1 = data_io.array2dataset(data1, datatypes)

hdf.create_table("Repres_Form", data1, dtype=datatypes)
hdf.append_entry("Repres_Form", data1)
cols = hdf.column_names("Repres_Form")


data2 = np.random.normal(size=(100, 33*600)); 
hdf.create_dataset("FPFH", data2)
hdf.append_entry("FPFH", data2)


tmp = np.random.normal(size=(32,32)); 

datatypes = np.dtype([('f_id', np.int32),('f', np.float32, (32*32))])
# datatypes = [('f_id', '<i4'), ('f', '<f4', (32* 32))]
data3 = np.array([[i, np.random.normal(size=32*32)] for i in range(100)], dtype=object)  # 1d+3d Data hybrid object
data3 = data_io.array2dataset(data3, datatypes)


# data3 = np.array([[i, tmp]for i in range(100)], dtype=[("f_id", int), ("f", h5.vlen_dtype(float))])
hdf.create_dataset("Feature_mass", data3)
hdf.append_entry("Feature_mass", data3)
print(hdf.data("Feature_mass")[0].dtype)

hdf.draw_structure()

dt1 = hdf.data("Repres_Form")[0].dtype

table = pd.DataFrame(hdf.data("Repres_Form")[:])

hdf.close()

table

Initialize datasets
Created Dataset: Repres_Form
Appended 100 entries to Repres_Form
Created Dataset: FPFH
Appended 100 entries to FPFH
Created Dataset: Feature_mass
Appended 100 entries to Feature_mass
[('f_id', '<i4'), ('f', '<f4', (1024,))]
####### HDF File Structure #######
$ /FPFH                : Shape-(200, 19800)
$ /Feature_mass        : Shape-(200,)
$ /Repres_Form         : Shape-(200,)
##### END HDF File Structure #####


,atom_number,carbon_number,donor_number,acceptor_number,positive_charge,negative_charge,pseudo_lj,pseudo_elec,surface_area,volume,mean_radius,convex_ratio
0,0,0,0,0,0.697333,-0.079331,1.475524,0.587813,-1.387170,-0.133004,0.788414,-0.278294
1,0,0,1,1,0.660080,-0.801780,0.451647,0.679890,0.639444,0.734185,1.870919,-2.109832
2,0,-1,-1,0,-0.763192,0.581175,-0.252742,-1.465508,-0.545294,1.114773,-0.387818,-0.150808
3,1,0,1,-1,-1.025112,1.491295,1.774307,-0.579074,0.607775,-0.629425,-0.213670,1.523773
4,1,0,-1,0,0.597104,0.698269,-0.020304,-0.980685,-0.094954,1.147845,0.439396,-1.448967
...,...,...,...,...,...,...,...,...,...,...,...,...
195,-2,-1,0,0,1.840170,0.798444,0.237008,1.853497,-1.236781,-0.547262,1.084265,0.525325
196,1,1,0,0,-0.211413,-0.001468,-1.881163,0.281422,0.026797,-0.460763,-1.496213,2.151600
197,0,0,0,0,0.171427,0.157518,-0.260131,-0.546909,-0.349130,0.220440,1.332984,-0.312485
198,0,0,1,0,2.664379,0.361938,-0.333593,-0.913620,-0.143810,1.680028,-1.330220,0.997636
